In [ ]:
import os
import time
import random
import pandas as pd
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed

client = OpenAI(
    api_key="",
    base_url=""
)

MODEL_NAME = ""

INPUT_PRICE = 2 / 1_000_000
OUTPUT_PRICE = 8 / 1_000_000

JUDGE_PROMPT_TEMPLATE = """
"""

def call_judge_llm(keyword, category, max_retries=3):
    keyword = keyword.strip()
    if not keyword:
        return "否", None

    system_message = {"role": "system", "content": JUDGE_PROMPT_TEMPLATE.format(keyword=keyword, category=category)}
    user_message = {"role": "user", "content": f"请判断关键词“{keyword}”是否属于“{category}”。"}

    messages = [system_message, user_message]

    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
            )
            usage = completion.usage
            answer = completion.choices[0].message.content.strip()

            if "是" in answer and "否" not in answer:
                return "是", usage
            elif "否" in answer:
                return "否", usage
            else:
                time.sleep(1)

        except Exception:
            time.sleep(min(2 ** attempt, 10) * random.uniform(0.8, 1.2))

    return "否", None


def judge_keywords(df, max_workers=10):
    categories = ["视觉关键词", "听觉关键词", "嗅觉关键词", "味觉关键词", "触觉关键词"]
    total_prompt_tokens = 0
    total_completion_tokens = 0

    keywords_with_category = []
    for _, row in df.iterrows():
        for cat in categories:
            original_text = row.get(cat, "")
            if not original_text or pd.isna(original_text):
                continue
            keywords = [kw.strip() for kw in original_text.split("、") if kw.strip()]
            for kw in keywords:
                keywords_with_category.append((kw, cat))

    keywords_with_category = list(set(keywords_with_category))

    def task(keyword_cat):
        kw, cat = keyword_cat
        res, usage = call_judge_llm(kw, cat)
        p_tokens = getattr(usage, "prompt_tokens", 0) if usage else 0
        c_tokens = getattr(usage, "completion_tokens", 0) if usage else 0
        return kw, cat, res, p_tokens, c_tokens

    results = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(task, kc): kc for kc in keywords_with_category}
        for future in as_completed(futures):
            kw, cat = futures[future]
            try:
                k, c, res, p, co = future.result()
                results[(k, c)] = res
                total_prompt_tokens += p
                total_completion_tokens += co
            except Exception:
                results[(kw, cat)] = "否"

    new_columns = {cat: [] for cat in categories}
    deleted_words_col = []

    for _, row in df.iterrows():
        deleted_words = []
        for cat in categories:
            original_text = row.get(cat, "")
            if not original_text or pd.isna(original_text):
                new_columns[cat].append("")
                continue

            keywords = [kw.strip() for kw in original_text.split("、") if kw.strip()]
            kept_keywords = []
            for kw in keywords:
                final_result = results.get((kw, cat), "否")
                if final_result == "是":
                    kept_keywords.append(kw)
                else:
                    deleted_words.append(f"{kw}（{cat}）")

            new_columns[cat].append("、".join(kept_keywords))

        deleted_words_col.append("、".join(deleted_words))

    for cat in categories:
        df[cat] = new_columns[cat]
    df["删除词（含原感官维度）"] = deleted_words_col

    return df, total_prompt_tokens, total_completion_tokens


def process_judge_excel(input_excel_path, output_excel_path):
    df = pd.read_excel(input_excel_path, dtype=str)
    df.fillna("", inplace=True)

    df_judged, prompt_tokens, completion_tokens = judge_keywords(df)
    df_judged.to_excel(output_excel_path, index=False)

    print(f"✅ 判定并保存完成：{output_excel_path}")
    return prompt_tokens, completion_tokens, os.path.basename(input_excel_path)


def process_all_excels_judge(input_folder, output_folder):
    excel_files = [os.path.join(root, f)
                   for root, _, files in os.walk(input_folder)
                   for f in files if f.lower().endswith(".xlsx")]
    print(f"共发现 {len(excel_files)} 个excel文件待处理判定\n")

    total_prompt_tokens = 0
    total_completion_tokens = 0
    file_stats = []

    start_time = time.time()
    elapsed_times = []

    for idx, excel_path in enumerate(excel_files, 1):
        file_start = time.time()

        print(f"[{idx}/{len(excel_files)}] 正在处理：{excel_path}")
        rel_path = os.path.relpath(excel_path, input_folder)
        out_path = os.path.join(output_folder, rel_path)
        os.makedirs(os.path.dirname(out_path), exist_ok=True)

        p_tokens, c_tokens, filename = process_judge_excel(excel_path, out_path)

        input_cost = p_tokens * INPUT_PRICE
        output_cost = c_tokens * OUTPUT_PRICE
        total_cost = input_cost + output_cost

        total_prompt_tokens += p_tokens
        total_completion_tokens += c_tokens

        elapsed = time.time() - file_start
        elapsed_times.append(elapsed)

        print(f"📁 {filename} Token消耗：输入 {p_tokens}，输出 {c_tokens}")
        print(f"💰 成本：输入 {input_cost:.6f} 元 + 输出 {output_cost:.6f} 元 = 合计 {total_cost:.6f} 元")

        if idx < len(excel_files):
            avg_time = sum(elapsed_times) / len(elapsed_times)
            remaining_files = len(excel_files) - idx
            est_remaining_time = avg_time * remaining_files / 60
            print(f"⏳ 预计剩余处理时间约 {est_remaining_time:.1f} 分钟\n")

        file_stats.append((filename, p_tokens, c_tokens, input_cost, output_cost, total_cost))

    print("\n✅ 判定阶段全部完成\n")

    print("📊 每个文件详细Token消耗与成本：")
    for f, p, c, ci, co, ct in file_stats:
        print(f"📄 {f:30} | 输入: {p:<7} | 输出: {c:<7} | 成本: {ct:.6f} 元")

    total_input_cost = total_prompt_tokens * INPUT_PRICE
    total_output_cost = total_completion_tokens * OUTPUT_PRICE
    total_cost = total_input_cost + total_output_cost

    total_elapsed = time.time() - start_time
    print(f"\n📈 总Token消耗：输入 {total_prompt_tokens}，输出 {total_completion_tokens}")
    print(f"💰 总成本：输入 {total_input_cost:.6f} 元 + 输出 {total_output_cost:.6f} 元 = 合计 {total_cost:.6f} 元")
    print(f"🕒 实际总耗时：{total_elapsed / 60:.1f} 分钟")

if __name__ == "__main__":
    input_folder = r""
    output_folder = r""
    process_all_excels_judge(input_folder, output_folder)
